In [1]:
from __future__ import annotations
from secuencia import Secuencia

from typing import NamedTuple
import inspect

def parent_module(f: callable) -> str: return f.__qualname__.split('.')[0]

def flatten(s):
    result = []
    for item in s:
        if isinstance(item, list):
            result.extend(flatten(item))
        else:
            result.append(item)
    return result

class Parametro(NamedTuple):
    nombre: str
    tipo: str

class Funcion(NamedTuple):
    nombre: str
    parametros: list[Parametro]
    resultado: str
    codigo: list[str]

    @staticmethod
    def from_callable(f: callable) -> Funcion:
        signatura = inspect.signature(f)
        return Funcion(nombre=f.__name__, 
                       resultado=signatura.return_annotation,
                       codigo=inspect.getsourcelines(f),
                       parametros=[Parametro(k,v.annotation if v.kind != inspect._ParameterKind.VAR_POSITIONAL else f"...{v.annotation}") for k,v in \
                                   inspect.signature(f).parameters.items()],
                        )

In [2]:
# Depende de github.com/joangq/tad-latex/tad.sty
class signaturas:
    @classmethod
    def latexify(cls, f: Funcion, symbol: str|None = None) -> str:
        nombre = symbol or '$'+f.nombre+'$' if len(f.nombre) == 1 else rf'\text{{{f.nombre}}}'
        
        resultado = f.resultado
        parametros = [p.tipo for p in f.parametros] + [resultado]

        for i,param in enumerate(parametros):
            p = str(param)
            if '...' in p:
                p = p.replace('...', '')
                p = rf'{p} $\times\dots\times$ {p}'
            parametros[i] = '{'+p+'}'
        
        return ''.join([rf'\signature{{{nombre}}}', *parametros])
        

In [3]:
TAD_Secu = Secuencia.get_tad()

In [4]:
from catthy import fun
from xpytex.utils import displaymath

In [5]:
def consume_multiple__(n, f, args, acc=None):
    L = ((1 if acc is not None else 0)+len(args))
    if L != n and L%(n-1) != 1: 
        raise Exception(f"Incompatible length {L} for {n} args. Expected {L+1} or {L-1}.")

    if acc is not None:
        offset = 0
    else:
        acc = args[0]
        offset = 1

    n = n-1
    for i in range(offset, len(args)-n+1+offset, n):
        acc = f(acc, *[args[i+j] for j in range(0,n)])
    
    return acc

def consume_multiple(f, args, acc=None):
    return consume_multiple__(len(inspect.signature(f).parameters), f, args, acc)

#consume_multiple(lambda a,b,c: rf'({a}? {b} : {c})', '123' ,'3')

In [6]:
def multop_to_varadic(multop, parent_module = None):
    if not parent_module:
        parent_module = parent_module(multop)
    
    def g(name: str, args: list[str], parent: list):
        if name != multop.__name__: return ''
        else:
            if parent == parent_module:
                return consume_multiple(multop, args)
            else:
                return consume_multiple(multop, args, parent)
            
    # @wraps also replaces __code__ with new varnames
    g.__name__ = multop.__name__
    g.__qualname__ = multop.__qualname__
    g.__doc__ = multop.__doc__
    g.__module__ = multop.__module__
    g.__annotations__ = multop.__annotations__
    g.__source__ = inspect.getsource(multop)
    #g.__wrapped__ = multop

    return g

def grammar_multop_to_varadic(cls, binop: str):
    return multop_to_varadic(getattr(cls, binop), getattr(cls, 'tad'))

#def TernaryIfElse(a,b,c): return rf'({a}? {b} : {c})'
#multop_to_varadic(TernaryIfElse, 'IfElse')('TernaryIfElse', ['1','2','3','4','5'], 'IfElse')

In [7]:
def binop_to_varadic(binop, parent_module = None):
    """Converts a binary operator to a varadic one to match the latexify Function Call facade."""
    if not parent_module:
        parent_module = parent_module(binop)
        
    def g(name, args, parent):
        if name != binop.__name__: return ''
        else:
            if parent == parent_module:
                return fun.foldl(binop, args)
            else:
                return fun.foldl(binop, args, parent)
            
    # @wraps also replaces __code__ with new varnames
    g.__name__ = binop.__name__
    g.__qualname__ = binop.__qualname__
    g.__doc__ = binop.__doc__
    g.__module__ = binop.__module__
    g.__annotations__ = binop.__annotations__
    g.__source__ = inspect.getsource(binop)
    #g.__wrapped__ = binop
    return g

def grammar_binop_to_varadic(cls, binop: str):
    return binop_to_varadic(getattr(cls, binop), getattr(cls, 'tad'))

In [8]:
def unop_to_varadic(unop, parent_module = None):
    """Converts a unary operator to a varadic one to match the latexify Function Call facade."""
    if not parent_module:
        parent_module = parent_module(unop)
        
    def g(name, args, parent):
        if name != unop.__name__: return ''
        if len(args) > 1:
            raise TypeError("More than one argument in unary operator.")
        
        if len(args) == 0:
            return unop(parent)
        else:
            return unop(args[0])

            
    # @wraps also replaces __code__ with new varnames
    g.__name__ = unop.__name__
    g.__qualname__ = unop.__qualname__
    g.__doc__ = unop.__doc__
    g.__module__ = unop.__module__
    g.__annotations__ = unop.__annotations__
    g.__source__ = inspect.getsource(unop)
    #g.__wrapped__ = unop
    return g

def grammar_unop_to_varadic(cls, unop: str):
    return unop_to_varadic(getattr(cls, unop), getattr(cls, 'tad'))

In [9]:
def varop_to_varadic(varop, parent_module = None):
    """Adapts a varadic single-argument operator to the latexify Function call facade"""
    if not parent_module:
        parent_module = parent_module(varop)
        
    def g(name, args, parent):
        if name != varop.__name__: return ''
        if parent != parent_module: return ''
        return varop(*args)
            
    # @wraps also replaces __code__ with new varnames
    g.__name__ = varop.__name__
    g.__qualname__ = varop.__qualname__
    g.__doc__ = varop.__doc__
    g.__module__ = varop.__module__
    g.__annotations__ = varop.__annotations__
    g.__source__ = inspect.getsource(varop)
    #g.__wrapped__ = varop
    return g

def grammar_varop_to_varadic(cls, varop: str):
    return varop_to_varadic(getattr(cls, varop), getattr(cls, 'tad'))

In [10]:
#TODO: Convert this into a decorator that converts the methods automatically.

# for k,v in CLASS.__dict__.items():
#     if v and isinstance(v, staticmethod):
#         f = getattr(CLASS, k)
#         ps = inspect.signature(f).parameters
#         n = len( ps )
#         if n == 2:
#             setattr(CLASS, k, grammar_binop_to_varadic(CLASS, k))
#         elif n == 1:
#             if list(ps.items())[0][1].kind is inspect._ParameterKind.VAR_POSITIONAL: # is varadic?
#                 setattr(CLASS, k, grammar_varop_to_varadic(CLASS, k))
#             else:
#                 setattr(CLASS, k, grammar_unop_to_varadic(CLASS, k))
#         #setattr(CLASS, k, staticmethod(ltxargs(v.__func__)))

class GrammarClass:
    """All grammar classes should have this as a metaclass."""
    __tad__: str

    # ADT Functions should be:
    # @staticmethod
    # def f(params... : str) -> str: ...
    
    @classmethod
    def latexify(cls, name: str, args: list[str], parent:None|str=None) -> str:
        return getattr(cls, name)(name, args, parent)

In [11]:
class secuencias:
    """Grammar for a sequence"""
    tad = 'Secuencia'
    @staticmethod
    def prim(s): return r'\text{prim}('+s+')'

    @staticmethod
    def esVacia(s): return r'\text{vacía?}('+s+')'

    @staticmethod
    def fin(s): return r'\text{fin}('+s+')'

    @staticmethod
    def vacia(): return r'\langle\rangle'

    @staticmethod
    def agregarAdelante(s, e): return e+r'\text{ }\bullet{}\text{ }'+s

    @staticmethod
    def concatenar(s, t): return s+r'\texttt{ \& }'+t

    @staticmethod
    def com(s): return r'\text{com}('+s+')'

    @staticmethod
    def esta(s, e): return r'\text{está}?('+e+', '+s+')'

    @staticmethod
    def agregarAtras(s, e): return s+r'\text{ }\circ{}\text{ }'+e

    @staticmethod
    def ult(s): return r'\text{ult}('+s+')'

    @staticmethod
    def long(s): return r'\text{long}('+s+')'

    @staticmethod
    def de(*xs): return r'\left\langle{}'+ ', '.join(xs) +r'\right\rangle{}'

    @classmethod
    def latexify(cls, name: str, args: list[str], parent:None|str=None) -> str:
        return getattr(cls, name)(name, args, parent) # Internally this could be simplified to
                                                      # (args, parent). So that class.method(args, parent)
                                                      # is the same as class.latexify(methodname, args, parent)

In [12]:
# TODO: Wrap this into... a wrapper
for k,v in secuencias.__dict__.items():
    if v and isinstance(v, staticmethod):
        f = getattr(secuencias, k)
        ps = inspect.signature(f).parameters
        n = len( ps )
        if n == 2:
            rule = grammar_binop_to_varadic(secuencias, k)
            setattr(secuencias, k, rule)
        elif n == 1:
            if list(ps.items())[0][1].kind is inspect._ParameterKind.VAR_POSITIONAL: # is varadic?
                setattr(secuencias, k, grammar_varop_to_varadic(secuencias, k))
            else:
                setattr(secuencias, k, grammar_unop_to_varadic(secuencias, k))
        #setattr(secuencias, k, staticmethod(ltxargs(v.__func__)))

In [13]:
secuencias.latexify('esVacia', [], '<abc>') == '\\text{vacía?}(<abc>)',\
secuencias.latexify('esVacia', '1', '<abc>') == '\\text{vacía?}(1)'

(True, True)

In [14]:
displaymath(
    secuencias.latexify('agregarAdelante', '1234', secuencias.latexify('de', ['a','b','c'], 'Secuencia'))
)

<IPython.core.display.Math object>

In [15]:
print(secuencias.latexify('agregarAdelante', '1234', secuencias.latexify('de', ['a','b','c'], 'Secuencia')))

4\text{ }\bullet{}\text{ }3\text{ }\bullet{}\text{ }2\text{ }\bullet{}\text{ }1\text{ }\bullet{}\text{ }\left\langle{}a, b, c\right\rangle{}


In [16]:
displaymath(
    secuencias.latexify('agregarAdelante', [secuencias.de('3','2','1'),'4','5'], 'Secuencia')
)

<IPython.core.display.Math object>

In [17]:
displaymath(
secuencias.latexify('esVacia', '1', '<abc>')
)

<IPython.core.display.Math object>

In [18]:
secuencias.latexify('agregarAtras', ["1","2"], 'Secuencia')

'1\\text{ }\\circ{}\\text{ }2'

In [19]:
secuencias.latexify('ult', '', '[1,2,3]')

'\\text{ult}([1,2,3])'

In [20]:
from functools import reduce
from operator import truediv

In [21]:
fs = flatten([list(v) for k,v in TAD_Secu.items()][:-1])
lx_fs = [''] * len(fs)
for i,multop in enumerate(fs):
    lx_fs[i] = (signaturas.latexify(Funcion.from_callable(multop))+'\par')

for lxf in lx_fs:
    print(lxf)

\signature{\text{esVacia}}{Secuencia[T]}{bool}\par
\signature{\text{fin}}{Secuencia[T]}{Secuencia[T]}\par
\signature{\text{prim}}{Secuencia[T]}{T}\par
\signature{\text{vacia}}{Secuencia[T]}\par
\signature{\text{agregarAdelante}}{Secuencia[T]}{T}{Secuencia[T]}\par
\signature{\text{concatenar}}{Secuencia[T]}{Secuencia[T]}{Secuencia[T]}\par
\signature{\text{com}}{Secuencia[T]}{Secuencia[T]}\par
\signature{\text{esta}}{Secuencia[T]}{T}{bool}\par
\signature{\text{agregarAtras}}{Secuencia[T]}{T}{Secuencia[T]}\par
\signature{\text{ult}}{Secuencia[T]}{T}\par
\signature{\text{long}}{Secuencia[T]}{int}\par
\signature{\text{de}}{T $\times\dots\times$ T}{Secuencia[T]}\par


In [22]:
traducir = {
"observers": "observadores",
"generators": "generadores",
"operations": "operaciones"
}

In [23]:
print(r'\begin{tad}{'+secuencias.tad+'}')
for k,v in TAD_Secu.items():
    if k == 'axioms': continue # skip
    print('\t'+r'\begin{'+traducir[k]+'}')
    for f in v:
        print('\t\t'+signaturas.latexify(Funcion.from_callable(f))+'\par')
    print('\t'+r'\end{'+traducir[k]+'}')
print(r'\end{tad}')

\begin{tad}{Secuencia}
	\begin{observadores}
		\signature{\text{esVacia}}{Secuencia[T]}{bool}\par
		\signature{\text{fin}}{Secuencia[T]}{Secuencia[T]}\par
		\signature{\text{prim}}{Secuencia[T]}{T}\par
	\end{observadores}
	\begin{generadores}
		\signature{\text{vacia}}{Secuencia[T]}\par
		\signature{\text{agregarAdelante}}{Secuencia[T]}{T}{Secuencia[T]}\par
	\end{generadores}
	\begin{operaciones}
		\signature{\text{concatenar}}{Secuencia[T]}{Secuencia[T]}{Secuencia[T]}\par
		\signature{\text{com}}{Secuencia[T]}{Secuencia[T]}\par
		\signature{\text{esta}}{Secuencia[T]}{T}{bool}\par
		\signature{\text{agregarAtras}}{Secuencia[T]}{T}{Secuencia[T]}\par
		\signature{\text{ult}}{Secuencia[T]}{T}\par
		\signature{\text{long}}{Secuencia[T]}{int}\par
		\signature{\text{de}}{T $\times\dots\times$ T}{Secuencia[T]}\par
	\end{operaciones}
\end{tad}
